In [47]:
import pandas as pd
import numpy as np

In [48]:
data = pd.read_csv("./data/dataset_TSMC2014_NYC_hawkes.csv")
data = data.drop(data.iloc[:,[0]],axis=1)
data = data.iloc[:10000]
data.head()


,uid,big_vc_name,minutes,hours
0,1,Restaurant,1771.366667,29.522778
1,1,Restaurant,5742.250000,95.704167
2,1,Indoor Entertainment,7220.333333,120.338889
3,1,Restaurant,7322.016667,122.033611
4,1,Restaurant,8540.716667,142.345278


In [49]:
from random import randint

def is_sorted(arr):
    if(all(arr[i] <= arr[i + 1] for i in range(len(arr)-1))): return True
    return False

types=list(set(data.big_vc_name))
type2idx={t:i for i,t in enumerate(types,start=1)}
idx2type={i:t for i,t in enumerate(types,start=1)}
data['type']=[type2idx[t] for t in data.big_vc_name]
data_processed=data[['uid','type','hours']]
data_list=[x for _, x in data_processed.groupby(data_processed.uid)] # S = s_1, s_2, ... , S_C 
seq2ptr={c:np.where(data_processed.uid==df.uid.iloc[0])[0][0] for c,df in enumerate(data_list,start=1)}
assert(is_sorted(data_list[randint(0,len(data_list)-1)]['hours'].tolist()))
assert(all([df.uid.iloc[0]==i for i,df in enumerate(data_list,start=1)]))
Ts=[df.hours.iloc[-1]-df.hours.iloc[0] for _,df in enumerate(data_list)] # T_1, T_2, ... , T_C
print(types,'\n')
print(data_processed.head(),'\n')
print(data_list)

['School', 'Smoke Shop', 'Food & Snack', 'Travel-Related Place', 'Residence', 'Hotel', 'Neighborhood', 'History, Museum & Arts', 'Convention Center', 'Office', 'Electronics', 'Medical-Related Place', 'Transportation', 'General Education Place', 'Restaurant', 'Place for Sports', 'Outdoor Entertainment', 'Vehicle-Related Place', 'Rest Area', 'Store, Market & Fair', 'Indoor Entertainment'] 

   uid  type       hours
0    1    15   29.522778
1    1    15   95.704167
2    1    21  120.338889
3    1    15  122.033611
4    1    15  142.345278 

[    uid  type        hours
0     1    15    29.522778
1     1    15    95.704167
2     1    21   120.338889
3     1    15   122.033611
4     1    15   142.345278
..  ...   ...          ...
91    1    13  3094.666111
92    1    15  3095.082778
93    1    21  6081.085556
94    1    15  6119.923056
95    1    13  6126.214722

[96 rows x 3 columns],      uid  type        hours
96     2     3    19.726944
97     2    20    21.600000
98     2    15   117.19

In [50]:
from math import pi,exp
alpha_S,alpha_G=10,100 # 10^-2 ~ 10^4, coefficients for regularization terms
M=96 # 24h/15min, number of basis function
T=24 # 24h in one day
w0=pi*M/T # parameter for basis functions
t_ms=[(m-1)*T/M for m in range(1,M+1)] # parameters for basis functions
bases=[lambda t,t_m=t_m:(t<=T)*exp(-((t-t_m)*w0)**2 / 2) for t_m in t_ms] # list of basis functions
U=len(types) # number of types
C=len(set(data.uid)) # number of sequences

In [51]:
import numpy as np
from math import sqrt,pi
from scipy.special import erf
from tqdm import tqdm

scaler=1
mus=np.abs(np.random.rand(U))*scaler # exogenous base intensity independent of the history
A=np.abs(np.random.rand(U,U,M))*scaler # A[u,v,m]=a_{uv}^m, where v->u

def cond_intensity(u:int,t:int,c:int,bases=bases,mus=mus,A=A,data_list=data_list):
    ''' 
    u: event type with range [1,U]
    t: time with range [0,T_c] in hours
    c: index of sequence with range [1,C] 
    '''
    data=data_list[c-1]
    data=data[data['hours']<t] # (uid, type, hours)
    coefs=A[u-1]
    lambda_t=mus[u-1]+np.sum([np.dot(coefs[tp-1],[bf(t-hr) for bf in bases]) for _,tp,hr in zip(data['uid'],data['type'],data['hours'])])
    return lambda_t

eps=1e-9
sqrt2=sqrt(2)
const1=sqrt(pi/2)/w0
mus_new=mus.copy()
A_new=A.copy()
print('A\n',A)
print('mu\n',mus)
while(True):
    while(True):
        print('iteration start...')
        # TODO: update mus, A
        print('calculating conditional intensities...')
        cond_intensities=[]
        for uid,tp,hr in tqdm(zip(data_processed['uid'],data_processed['type'],data_processed['hours'])):
            cond_intensities.append(cond_intensity(tp,hr,uid))
        # print(cond_intensities)
        print('conditional intensities updated!')
        print('calculating mu...')
        p_sum=np.zeros(U)
        for i,tp in tqdm(enumerate(data_processed['type'])):
            p_sum[tp-1]+=mus[tp-1]/cond_intensities[i]
        T_sum=np.sum(Ts)
        mus_new=p_sum/T_sum # update mus
        print('mu updated!')
        
        print('calculating A_coef...')
        A_coef=alpha_G/np.apply_along_axis(np.linalg.norm,2,A)
        # print(A_coef)
        print('A_coef finished!')
        print('calculating B_coef...')
        B_coef=np.ones((U,M))*alpha_S
        for uid,tp,hr in tqdm(zip(data_processed['uid'],data_processed['type'],data_processed['hours'])):
            for m in range(M):
                # re := integrate.quad(bases[m],0,Ts[uid-1]-hr)[0]
                t,tm=Ts[uid-1]-hr,t_ms[m]
                re=const1*(erf(w0*tm/sqrt2) + (t<T)*erf((t-tm)*w0/sqrt2) + (t>=T)*erf((T-tm)*w0/sqrt2))
                B_coef[tp-1][m]+=re
        # print(B_coef)
        print('B_coef finished!')
        print('calculating C_coef...')
        C_coef=np.zeros((U,U,M))
        for c in tqdm(range(C)):
            data_df=data_list[c]
            df_len=len(data_df)
            for idx1,uid1,tp1,hr1 in zip(list(range(df_len)),data_df['uid'],data_df['type'],data_df['hours']):
                for idx2,uid2,tp2,hr2 in zip(list(range(idx1)),data_df['uid'],data_df['type'],data_df['hours']):
                    cond_int=cond_intensities[seq2ptr[c+1]+idx1]
                    for m in range(M):
                        C_coef[tp1-1][tp2-1][m]-=bases[m](hr1-hr2)*A[tp1-1][tp2-1][m]/cond_int
            
        for c in range(C):
            data_list[c]['hours']-data_list[c]['hours']
        # print(C_coef)
        print('C_coef finished!')
        print('calculating A...')
        for tp1 in range(U):
            for tp2 in range(U):
                for m in range(M):
                    A_tmp,B_tmp,C_tmp=A_coef[tp1][tp2],B_coef[tp2][m],C_coef[tp1][tp2][m]
                    re=(-B_tmp+sqrt(B_tmp**2-4*A_tmp*C_tmp))/(2*A_tmp)
                    A_new[tp1][tp2][m]=re
        print('A updated!')
        print('A:')
        print(A_new)
        print('mu:')
        print(mus_new)
        print('iteration ends...')
        if np.allclose(A, A_new) and np.allclose(mus,mus_new):
            A,mus=A_new,mus_new
            break
        else: 
            A,mus=A_new,mus_new
            continue
    # TODO: sparse-group-lasso
    break

A
 [[[0.63577858 0.62976509 0.77150595 ... 0.47055687 0.53720038 0.44411359]
  [0.33537513 0.38598787 0.83167831 ... 0.41779426 0.42046905 0.96918985]
  [0.8423799  0.33262656 0.78633762 ... 0.36011489 0.52463521 0.77863308]
  ...
  [0.87537511 0.56214544 0.07259575 ... 0.37528723 0.57233349 0.49064374]
  [0.89578745 0.67155019 0.45102472 ... 0.26233827 0.26109647 0.83046517]
  [0.96952169 0.23139558 0.59578209 ... 0.39736923 0.50881318 0.59203285]]

 [[0.7822812  0.78144487 0.56438721 ... 0.61367114 0.74968828 0.29398011]
  [0.76130403 0.57553438 0.7608886  ... 0.08376736 0.94843985 0.42700372]
  [0.90728181 0.45413557 0.27082398 ... 0.68086111 0.11214158 0.67234549]
  ...
  [0.33461882 0.25155105 0.47493742 ... 0.76067823 0.41808471 0.60579664]
  [0.34882632 0.4130911  0.11818549 ... 0.17781572 0.30108087 0.61257147]
  [0.20605703 0.37160958 0.20064727 ... 0.04896153 0.95994038 0.62214424]]

 [[0.85245196 0.74283295 0.03923372 ... 0.73320221 0.35054327 0.47140461]
  [0.23451236 0.065

10000it [01:13, 135.24it/s]


conditional intensities updated!
calculating mu...


10000it [00:00, 500489.71it/s]


mu updated!
calculating A_coef...
A_coef finished!
calculating B_coef...


...

mu updated!
calculating A_coef...
A_coef finished!
calculating B_coef...


10000it [00:08, 1116.35it/s]


B_coef finished!
calculating C_coef...


  7%|▋         | 5/67 [00:10<02:09,  2.09s/it]


KeyboardInterrupt: 

In [ ]:
from random import randint
import numpy as np
c,m=0,0
tm=t_ms[m]
arr=data_list[c]['hours'].to_numpy().reshape(-1,1)
diff=arr-arr.T
n1,n2=randint(0,len(arr)-1),randint(0,len(arr)-1)
assert(diff[n1][n2]==arr[n1]-arr[n2])
re=np.exp(-((diff-tm)*w0)**2 / 2)*(diff<=T)
arr_type=data_list[c]['type'].to_numpy().reshape(-1,1)
rows=np.repeat(arr_type,len(arr_type),axis=1)
cols=np.repeat(arr_type.T,len(arr_type),axis=0)
para=A[rows,cols]
n1,n2,n3=randint(0,len(arr)-1),randint(0,len(arr)-1),randint(0,M-1)
print(para[n1,n2,n3])
print(A[arr_type[n1],arr_type[n2],n3][0])
assert(para[n1,n2,n3]==A[arr_type[n1],arr_type[n2],n3])
para[:,:,m]